# Non-detuned Ramsey

In [ ]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

Check the RX and RX/2 * 2 are the same

In [ ]:
from qibolab.pulses import PulseSequence

sequence = PulseSequence()
qd_pulse = Device.create_RX_pulse(0, start=0)
ro_pulse = Device.create_qubit_readout_pulse(0, start=qd_pulse.finish)
sequence.add(qd_pulse)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

In [ ]:
from qibolab.pulses import PulseSequence

sequence = PulseSequence()
qd_pulse = Device.create_RX90_pulse(0, start=0)
qd_pulse2 = Device.create_RX90_pulse(0, start=qd_pulse.finish) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=qd_pulse2.finish)
sequence.add(qd_pulse)
sequence.add(qd_pulse2)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

In [ ]:
from qibocal.data import DataUnits
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np

def Ramsey(
    platform,
    qubit: int,
    delay_between_pulses_start,
    delay_between_pulses_end,
    delay_between_pulses_step,
    software_averages,
):
    
    platform.reload_settings()
    # platform.native_gates["single_qubit"][qubit]["RX"]["amplitude"] = platform.native_gates["single_qubit"][qubit]["RX"]["amplitude"]/2 
    # platform.instruments["shfqc_qc"]["settings"]["if_frequency"] = platform.instruments["shfqc_qc"]["settings"]["if_frequency"] + 1_000_000
    platform.apply_settings()
    
    data = DataUnits(name=f"data_q{qubit}", quantities={"wait": "ns", "t_max": "ns"})
    
    # sampling_rate = platform.sampling_rate
    # qubit_freq = platform.characterization["single_qubit"][qubit]["qubit_freq"]

    RX90_pulse1 = platform.create_RX90_pulse(qubit, start=0)
    RX90_pulse2 = platform.create_RX90_pulse(qubit, start=RX90_pulse1.finish)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=RX90_pulse2.finish)

    sequence = PulseSequence()
    sequence.add(RX90_pulse1)
    sequence.add(RX90_pulse2)
    sequence.add(ro_pulse)

    waits = np.arange(
        delay_between_pulses_start,
        delay_between_pulses_end,
        delay_between_pulses_step,
    )


    for _ in range(software_averages):
        for wait in waits:
            
            RX90_pulse2.start = RX90_pulse1.finish + wait
            ro_pulse.start = RX90_pulse2.finish

            msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)
            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[deg]": phase,
                "wait[ns]": wait,
                "t_max[ns]": delay_between_pulses_end,
            }
            
            data.add(results)
            
    # Plot simulated output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For Ramsey oscillations
def time_msr(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = DataUnits()

    fig = make_subplots(
        rows=1,
        cols=1,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=("MSR (V)",),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("wait", "ns"),
            y=data.get_values("MSR", "uV"),
            name="Ramsey",
        ),
        row=1,
        col=1,
    )

    # # add fitting

    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Time (ns)",
        yaxis_title="MSR (uV)",
    )
    return fig


In [ ]:
data = Ramsey(
    Device,
    0,
    delay_between_pulses_start = 0,
    delay_between_pulses_end = 15000,
    delay_between_pulses_step = 1000,
    software_averages = 1,
)

time_msr(data)

In [ ]:
from scipy.optimize import curve_fit

from qibocal.config import log
from qibocal.data import Data
from qibocal.fitting.utils import parse, ramsey

def ramsey_fit(data, x, y, qubit, qubit_freq, sampling_rate, offset_freq, labels):

    data_fit = Data(
        name=f"fit_q{qubit}",
        quantities=[
            "popt0",
            "popt1",
            "popt2",
            "popt3",
            "popt4",
            labels[0],
            labels[1],
            labels[2],
        ],
    )

    time = data.get_values(*parse(x))
    voltages = data.get_values(*parse(y))

    pguess = [
        np.mean(voltages.values),
        np.max(voltages.values) - np.min(voltages.values),
        0.5 / time.values[np.argmax(voltages.values)],  # ARGMAX FOR CAVITIES !!!
        np.pi / 2,
        500e-9,
    ]

    try:
        popt, pcov = curve_fit(
            ramsey, time.values, voltages.values, p0=pguess, maxfev=2000000
        )
        delta_fitting = popt[2]
        delta_phys = int((delta_fitting * sampling_rate) - offset_freq)
        corrected_qubit_frequency = int(qubit_freq + delta_phys)
        t2 = 1.0 / popt[4]
    except:
        log.warning("The fitting was not succesful")
        return data_fit

    data_fit.add(
        {
            "popt0": popt[0],
            "popt1": popt[1],
            "popt2": popt[2],
            "popt3": popt[3],
            "popt4": popt[4],
            labels[0]: delta_phys,
            labels[1]: corrected_qubit_frequency,
            labels[2]: t2,
        }
    )
    return data_fit

In [ ]:
data_fit = ramsey_fit(
data,
x="wait[ns]",
y="MSR[uV]",
qubit=0,
qubit_freq= Device.characterization["single_qubit"][0]["qubit_freq"] ,
sampling_rate= Device.settings["sampling_rate"] ,
offset_freq=0,
labels=[
    "delta_frequency",
    "corrected_qubit_frequency",
    "t2",
],
)

Don't trust 100 % the fit got the corrected frequency

In [ ]:
data_fit.df

------------------------------------

# Ramsey Frequency Detuned

In [ ]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

In [ ]:
from qibocal.data import DataUnits
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np


def ramsey_frequency_detuned(
    platform ,
    qubit: int,
    t_start,
    t_end,
    t_step,
    n_osc,
):
    
    platform.reload_settings()
    platform.native_gates["single_qubit"][qubit]["RX"]["amplitude"] = platform.native_gates["single_qubit"][qubit]["RX"]["amplitude"]/2 
    platform.apply_settings()
    
    sampling_rate = platform.settings["sampling_rate"]

    data = DataUnits(name=f"data_q{qubit}", quantities={"wait": "ns", "t_max": "ns"})

    RX90_pulse1 = platform.create_RX90_pulse(qubit, start=0)
    RX90_pulse2 = platform.create_RX90_pulse(qubit, start=RX90_pulse1.finish)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=RX90_pulse2.finish)

    sequence = PulseSequence()
    sequence.add(RX90_pulse1)
    sequence.add(RX90_pulse2)
    sequence.add(ro_pulse)

    # runcard_qubit_freq = platform.characterization["single_qubit"][qubit]["qubit_freq"]
    # # runcard_T2 = platform.characterization["single_qubit"][qubit]["T2"]
    # intermediate_freq = platform.settings["native_gates"]["single_qubit"][qubit]["RX"][
    #     "frequency"
    # ]

    # current_qubit_freq = runcard_qubit_freq

    # # FIXME: Waiting to be able to pass qpucard to qibolab
    # platform.ro_port[qubit].lo_frequency = (
    #     platform.characterization["single_qubit"][qubit]["resonator_freq"]
    #     - ro_pulse.frequency
    # )
    # platform.qd_port[qubit].lo_frequency = (
    #     platform.characterization["single_qubit"][qubit]["qubit_freq"]
    #     - RX90_pulse1.frequency
    # )

    # plat`form.qd_port[qubit].lo_frequency = current_qubit_freq - intermediate_freq
    offset_freq = n_osc / t_end * sampling_rate  # Hz
    
    Device.reload_settings()
    Device.instruments["shfqc_qc"]["settings"]["if_frequency"] = Device.instruments["shfqc_qc"]["settings"]["if_frequency"] + offset_freq
    Device.apply_settings()
    
    
    t_range = np.arange(t_start, t_end, t_step)
    for wait in t_range:
        # if count % points == 0 and count > 0:
        #     yield data
        #     yield ramsey_fit(
        #         data,
        #         x="wait[ns]",
        #         y="MSR[uV]",
        #         qubit=qubit,
        #         qubit_freq=current_qubit_freq,
        #         sampling_rate=sampling_rate,
        #         offset_freq=offset_freq,
        #         labels=[
        #             "delta_frequency",
        #             "corrected_qubit_frequency",
        #             "t2",
        #         ],
        #     )
        RX90_pulse2.start = RX90_pulse1.finish + wait
        
        ro_pulse.start = RX90_pulse2.finish

        msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)
        
        results = {
            "MSR[V]": msr,
            "i[V]": i,
            "q[V]": q,
            "phase[rad]": phase,
            "wait[ns]": wait,
            "t_max[ns]": t_end,
        }
        data.add(results)

    # # # Fitting
    # data_fit = ramsey_fit(
    #     data,
    #     x="wait[ns]",
    #     y="MSR[uV]",
    #     qubit=qubit,
    #     qubit_freq=current_qubit_freq,
    #     sampling_rate=sampling_rate,
    #     offset_freq=offset_freq,
    #     labels=[
    #         "delta_frequency",
    #         "corrected_qubit_frequency",
    #         "t2",
    #     ],
    # )

    # new_t2 = data_fit.get_values("t2")
    # corrected_qubit_freq = data_fit.get_values("corrected_qubit_frequency")

    # # if ((new_t2 * 3.5) > t_max):
    # if (new_t2 > current_T2).bool() and len(t_end) > 1:
    #     current_qubit_freq = int(corrected_qubit_freq)
    #     current_T2 = new_t2
    #     data = DataUnits(
    #         name=f"data_q{qubit}", quantities={"wait": "ns", "t_max": "ns"}
    #     )
    # else:
    #     corrected_qubit_freq = int(current_qubit_freq)
    #     new_t2 = current_T2
    #     break

    return data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For Ramsey oscillations
def time_msr(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = DataUnits()

    fig = make_subplots(
        rows=1,
        cols=1,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=("MSR (V)",),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("wait", "ns"),
            y=data.get_values("MSR", "uV"),
            name="Ramsey",
        ),
        row=1,
        col=1,
    )

    # # add fitting

    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Time (ns)",
        yaxis_title="MSR (uV)",
    )
    return fig


In [ ]:
data_detuned = ramsey_frequency_detuned(
    Device,
    qubit = 0,
    t_start = 0, 
    t_end = 15000, #t_end (optimal) = 3.5 * T2
    t_step = 1000,
    n_osc = 4
)

time_msr(data_detuned)

In [ ]:
t_end = 15000 #t_end (optimal) = 3.5 * T2
n_osc = 4

offset_freq = n_osc / t_end * 2e9
offset_freq

In [ ]:
from scipy.optimize import curve_fit

from qibocal.config import log
from qibocal.data import Data
from qibocal.fitting.utils import parse, ramsey

def ramsey_fit(data, x, y, qubit, qubit_freq, sampling_rate, offset_freq, labels):

    data_fit = Data(
        name=f"fit_q{qubit}",
        quantities=[
            "popt0",
            "popt1",
            "popt2",
            "popt3",
            "popt4",
            labels[0],
            labels[1],
            labels[2],
        ],
    )

    time = data.get_values(*parse(x))
    voltages = data.get_values(*parse(y))

    pguess = [
        np.mean(voltages.values),
        np.max(voltages.values) - np.min(voltages.values),
        0.5 / time.values[np.argmax(voltages.values)], # AGRMAX FOR CAVITIES !!!
        np.pi / 2,
        500e-9,
    ]

    try:
        popt, pcov = curve_fit(
            ramsey, time.values, voltages.values, p0=pguess, maxfev=2000000
        )
        delta_fitting = popt[2]
        delta_phys = int((delta_fitting * sampling_rate) - offset_freq)
        corrected_qubit_frequency = int(qubit_freq + delta_phys)
        t2 = 1.0 / popt[4]
    except:
        log.warning("The fitting was not succesful")
        return data_fit

    data_fit.add(
        {
            "popt0": popt[0],
            "popt1": popt[1],
            "popt2": popt[2],
            "popt3": popt[3],
            "popt4": popt[4],
            labels[0]: delta_phys,
            labels[1]: corrected_qubit_frequency,
            labels[2]: t2,
        }
    )
    return data_fit

In [ ]:
data_fit = ramsey_fit(
data_detuned,
x="wait[ns]",
y="MSR[uV]",
qubit=0,
qubit_freq= Device.characterization["single_qubit"][0]["qubit_freq"] ,
sampling_rate= Device.settings["sampling_rate"] ,
offset_freq= offset_freq ,
labels=[
    "delta_frequency",
    "corrected_qubit_frequency",
    "t2",
],
)

Don't trust 100 % the fit corrected frequency

In [ ]:
data_fit.df

----------------------------------------------------

# Ramsey Phase Detuned

In [ ]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

In [ ]:
from qibolab.pulses import PulseSequence

sequence = PulseSequence()
qd_pulse = Device.create_RX_pulse(0, start=0)
ro_pulse = Device.create_qubit_readout_pulse(0, start=qd_pulse.finish)
sequence.add(qd_pulse)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

In [ ]:
from qibolab.pulses import PulseSequence

sequence = PulseSequence()
qd_pulse = Device.create_RX90_pulse(0, start=0)
qd_pulse2 = Device.create_RX90_pulse(0, start=qd_pulse.finish) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=qd_pulse2.finish)
sequence.add(qd_pulse)
sequence.add(qd_pulse2)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

In [ ]:
from qibocal.data import DataUnits, Data
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np


def ramsey_phase_detuned(
    platform ,
    qubit: int,
    t_start,
    t_end,
    t_step,
    n_osc,
):
    
    platform.reload_settings()
    platform.apply_settings()
    
    sampling_rate = platform.settings["sampling_rate"]

    data = DataUnits(name=f"data_q{qubit}", quantities={"wait": "ns", "t_max": "ns"})

    RX90_pulse1 = platform.create_RX90_pulse(qubit, start=0)
    RX90_pulse2 = platform.create_RX90_pulse(qubit, start=RX90_pulse1.finish)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=RX90_pulse2.finish)

    sequence = PulseSequence()
    sequence.add(RX90_pulse1)
    sequence.add(RX90_pulse2)
    sequence.add(ro_pulse)

    # runcard_qubit_freq = platform.characterization["single_qubit"][qubit]["qubit_freq"]
    # # runcard_T2 = platform.characterization["single_qubit"][qubit]["T2"]
    # intermediate_freq = platform.settings["native_gates"]["single_qubit"][qubit]["RX"][
    #     "frequency"
    # ]

    # current_qubit_freq = runcard_qubit_freq

    # # FIXME: Waiting to be able to pass qpucard to qibolab
    # platform.ro_port[qubit].lo_frequency = (
    #     platform.characterization["single_qubit"][qubit]["resonator_freq"]
    #     - ro_pulse.frequency
    # )
    # platform.qd_port[qubit].lo_frequency = (
    #     platform.characterization["single_qubit"][qubit]["qubit_freq"]
    #     - RX90_pulse1.frequency
    # )

    # platform.qd_port[qubit].lo_frequency = current_qubit_freq - intermediate_freq
    offset_freq = n_osc / t_end * sampling_rate  # Hz
    
    t_range = np.arange(t_start, t_end, t_step)
    for wait in t_range:
        # if count % points == 0 and count > 0:
        #     yield data
        #     yield ramsey_fit(
        #         data,
        #         x="wait[ns]",
        #         y="MSR[uV]",
        #         qubit=qubit,
        #         qubit_freq=current_qubit_freq,
        #         sampling_rate=sampling_rate,
        #         offset_freq=offset_freq,
        #         labels=[
        #             "delta_frequency",
        #             "corrected_qubit_frequency",
        #             "t2",
        #         ],
        #     )
        RX90_pulse2.start = RX90_pulse1.finish + wait
        
        RX90_pulse2.relative_phase = (
            (RX90_pulse2.start / sampling_rate) * (2 * np.pi) * (-offset_freq)
        )
        
        ro_pulse.start = RX90_pulse2.finish

        msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)
        
        results = {
            "MSR[V]": msr,
            "i[V]": i,
            "q[V]": q,
            "phase[rad]": phase,
            "wait[ns]": wait,
            "t_max[ns]": t_end,
        }
        data.add(results)

    # # # Fitting
    # data_fit = ramsey_fit(
    #     data,
    #     x="wait[ns]",
    #     y="MSR[uV]",
    #     qubit=qubit,
    #     qubit_freq=current_qubit_freq,
    #     sampling_rate=sampling_rate,
    #     offset_freq=offset_freq,
    #     labels=[
    #         "delta_frequency",
    #         "corrected_qubit_frequency",
    #         "t2",
    #     ],
    # )

    # new_t2 = data_fit.get_values("t2")
    # corrected_qubit_freq = data_fit.get_values("corrected_qubit_frequency")

    # # if ((new_t2 * 3.5) > t_max):
    # if (new_t2 > current_T2).bool() and len(t_end) > 1:
    #     current_qubit_freq = int(corrected_qubit_freq)
    #     current_T2 = new_t2
    #     data = DataUnits(
    #         name=f"data_q{qubit}", quantities={"wait": "ns", "t_max": "ns"}
    #     )
    # else:
    #     corrected_qubit_freq = int(current_qubit_freq)
    #     new_t2 = current_T2
    #     break

    return data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For Ramsey oscillations
def time_msr(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = DataUnits()

    fig = make_subplots(
        rows=1,
        cols=1,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=("MSR (V)",),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("wait", "ns"),
            y=data.get_values("MSR", "uV"),
            name="Ramsey",
        ),
        row=1,
        col=1,
    )

    # # add fitting

    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Time (ns)",
        yaxis_title="MSR (uV)",
    )
    return fig


In [ ]:
t_end = 20000 #t_end (optimal) = 3.5 * T2
n_osc = 0

data_Detuned = ramsey_phase_detuned(
    Device,
    qubit = 0,
    t_start = 0, 
    t_end = t_end,
    t_step = 100,
    n_osc = n_osc
)

time_msr(data_Detuned)

In [ ]:
offset_freq = n_osc / t_end * 2e9
offset_freq

In [ ]:
from scipy.optimize import curve_fit

from qibocal.config import log
from qibocal.data import Data
from qibocal.fitting.utils import parse, ramsey

def ramsey_fit(data, x, y, qubit, qubit_freq, sampling_rate, offset_freq, labels):

    data_fit = Data(
        name=f"fit_q{qubit}",
        quantities=[
            "popt0",
            "popt1",
            "popt2",
            "popt3",
            "popt4",
            labels[0],
            labels[1],
            labels[2],
        ],
    )

    time = data.get_values(*parse(x))
    voltages = data.get_values(*parse(y))

    pguess = [
        np.mean(voltages.values),
        np.max(voltages.values) - np.min(voltages.values),
        0.5 / time.values[np.argmax(voltages.values)], # AGRMAX FOR CAVITIES !!!
        np.pi / 2,
        500e-9,
    ]

    try:
        popt, pcov = curve_fit(
            ramsey, time.values, voltages.values, p0=pguess, maxfev=2000000
        )
        delta_fitting = popt[2]
        delta_phys = int((delta_fitting * sampling_rate) - offset_freq)
        corrected_qubit_frequency = int(qubit_freq + delta_phys)
        t2 = 1.0 / popt[4]
    except:
        log.warning("The fitting was not succesful")
        return data_fit

    data_fit.add(
        {
            "popt0": popt[0],
            "popt1": popt[1],
            "popt2": popt[2],
            "popt3": popt[3],
            "popt4": popt[4],
            labels[0]: delta_phys,
            labels[1]: corrected_qubit_frequency,
            labels[2]: t2,
        }
    )
    return data_fit

In [ ]:
data_Fit = ramsey_fit(
data_Detuned,
x="wait[ns]",
y="MSR[uV]",
qubit=0,
qubit_freq= Device.characterization["single_qubit"][0]["qubit_freq"] ,
sampling_rate= Device.settings["sampling_rate"] ,
offset_freq= offset_freq ,
labels=[
    "delta_frequency",
    "corrected_qubit_frequency",
    "t2",
],
)

Don't trust 100 % the fit corrected frequency

In [ ]:
data_Fit.df